# 05_generate_nvz

In [1]:
import os

import numpy as np
from PIL import Image

In [2]:
def process_data(img_size, boxes):
    """ Box preprocessing: based on two diagonal coordinates convert box info to boxcenter_x, boxcenter_y, w, h
    and find the maximum number of boxes then do padding for all the boxes based on the maximum #boxes
    :param boxes: array with pure box diagonal coordinates info from train_uec100.txt
    :return:
    """
    # Original boxes stored as 1D list of class, x_min, y_min, x_max, y_max.
    boxes = [box.reshape((-1, 5)) for box in boxes]

    # Get box parameters as x_center, y_center, box_width, box_height, class.
    boxes_xy = [0.5 * (box[:, 3:5] + box[:, 1:3]) for box in boxes]
    boxes_wh = [box[:, 3:5] - box[:, 1:3] for box in boxes]
    boxes_xy = [boxxy / img_size for boxxy in boxes_xy]
    boxes_wh = [boxwh / img_size for boxwh in boxes_wh]
    boxes = [np.concatenate((boxes_xy[i], boxes_wh[i], box[:, 0:1]), axis=1) for i, box in enumerate(boxes)]

    # find the max number of boxes
    max_boxes = 0
    for box in boxes:
        if box.shape[0] > max_boxes:
            max_boxes = box.shape[0]

    # add zero pad for training
    for i, box in enumerate(boxes):
        if box.shape[0] < max_boxes:
            zero_padding = np.zeros((max_boxes - box.shape[0], 5), dtype=np.float32)
            boxes[i] = np.vstack((box, zero_padding))

    return np.array(boxes)

In [3]:
def create_label_dict(class_path):
    print('\n-> creating dictinary for labels...\n')
    label_dict = {}
    with open(class_path) as f:
        class_names = f.readlines()
    for i in range(0, len(class_names)):
        label_dict[class_names[i][:-1]] = i
    return label_dict

In [5]:
def txt2data(txt_path):
    """ Read train_uec100.txt file and convert to image_data array
    :return: image_data array with ['Volumes/JS/UECFOOD100_JS/1/1.jpg', [0,0,143,370,486]] kind of entries
    """
    print('\n-> converting txt info to data...\n')

    # Read train_uec100.txt file and save to a dict with directory as the key, bbox as value
    with open(txt_path, 'r') as f:
        entries = f.readlines()
        out = {}
        for i, entry in enumerate(entries):
            if i > 0:  # skip header
                entry = entry[:-1].split(' ')
                assert Image.open(entry[0]).size == (800, 600)  # after preprocessing size should be exactly 800,600
                entry[2] = ' '.join(entry[2:])
                entry[1] = str(int(entry[1]) - 1) + ' ' + entry[2]  # YOLO requires category id starts from 0 not 1
                entry = entry[:2]
                if entry[0] in out.keys():
                    out[entry[0]].append(entry[1])
                else:
                    out[entry[0]] = [entry[1]]

    # Save img directory with bbox info from out dict to image_data array
    image_data = list()
    index = 0
    for k, v in out.items():
        image_data.append([k])
        for i in v:
            image_data[index].append(i)
        index += 1

    # Convert string to int or float and save in image_data array again
    for no, entry in enumerate(image_data):
        for i, box in enumerate(entry):
            if i != 0:      # skip img path
                box = box.split(' ')
                box[0] = int(box[0])  # convert class name to numbers (0~)

                for k in range(1, 5):  # Change box boundaries from str to int
                    box[k] = int(float(box[k]))

                image_data[no][i] = box
    return image_data

In [6]:
def load_images(image_data):
    """ Load images based on their directory in the image_data array and save them in images then return
    :param image_data: acquired from txt2data()
    :return: images with each img info with shape(600, 800, 3) for each
    """
    print('\n -> Reading imgs and saving to array images...\n')
    images = []
    boxes = np.array([np.array(image_data[i][1:]) for i in range(np.array(image_data).shape[0])])
    image_data = np.array(image_data)
    boxes = process_data(img_size, boxes)
    detectors_mask, matching_true_boxes = get_detector_mask(boxes, anchors)

    for i, data in enumerate(image_data):
        img = Image.open(os.path.join(data[0]))
        assert img.size == (800, 600)
        img = np.array(img, dtype=np.uint8)
        images.append(img)
        boxes = np.array(image_data[i][1:])
        boxes = np.array(boxes)
    return images

In [7]:
def images2npv(images, image_data, shuffle=False):
    """ Save image info and box info to npv file
    :param images: image
    :param image_data:
    :param shuffle: if shuffle or not (data has been shuffled in during preprocessing)
    :return:
    """
    print('\n -> converting image info to npv file...\n')
    images = np.array(images, dtype=np.uint8)
    image_data = [np.array(image_data[i][1:]) for i in range(images.shape[0])]
    image_data = np.array(image_data)

    # shuffle dataset
    if shuffle:
        np.random.seed(13)
        indices = np.arange(len(images))
        np.random.shuffle(indices)
        images, image_data = images[indices], image_data[indices]
    print('dataset contains {} images'.format(images.shape[0]))
    np.savez('UECFOOD100npv_JS', image=images, boxes=image_data)
    print('npz file has been generated and saved as UECFOOD100npv_JS.npz')

In [8]:
def get_detector_mask(boxes, anchors):
    detectors_mask = [0 for i in range(len(boxes))]
    matching_true_boxes = [0 for i in range(len(boxes))]
    for i, box in enumerate(boxes):
        detectors_mask[i], matching_true_boxes[i] = preprocess_true_boxes(box, anchors, [416, 416])

    return np.array(detectors_mask), np.array(matching_true_boxes)

In [9]:
def preprocess_true_boxes(true_boxes, anchors, image_size):
    """Find detector in YOLO where ground truth box should appear

    Parameters
    ----------
    true_boxes : array
        List of ground truth boxes in form of relative x, y, w, h, class.
        Relative coordinates are in the range [0, 1] indicating a percentage
        of the original image dimensions.
    anchors : array
        List of anchors in form of w, h.
        Anchors are assumed to be in the range [0, conv_size] where conv_size
        is the spatial dimension of the final convolutional features.
    image_size : array-like
        List of image dimensions in form of h, w in pixels.

    Returns
    -------
    detectors_mask : array
        0/1 mask for detectors in [conv_height, conv_width, num_anchors, 1]
        that should be compared with a matching ground truth box.
    matching_true_boxes: array
        Same shape as detectors_mask with the corresponding ground truth box
        adjusted for comparison with predicted parameters at training time.
    """
    height, width = image_size
    num_anchors = len(anchors)

    # Downsampling factor of 5x 2-stride max_pools == 32.
    assert height % 32 == 0,    'Image sizes in YOLO_v2 must be multiples of 32.'
    assert width % 32 == 0,     'Image sizes in YOLO_v2 must be multiples of 32.'
    conv_height = height // 32
    conv_width = width // 32
    num_box_params = true_boxes.shape[1]
    detectors_mask = np.zeros(
        (conv_height, conv_width, num_anchors, 1), dtype=np.float32)
    matching_true_boxes = np.zeros(
        (conv_height, conv_width, num_anchors, num_box_params),
        dtype=np.float32)

    for box in true_boxes:
        # scale box to convolutional feature spatial dimensions
        box_class = box[4:5]
        box = box[0:4] * np.array(
            [conv_width, conv_height, conv_width, conv_height])
        i = np.floor(box[1]).astype('int')
        j = np.floor(box[0]).astype('int')
        if j >= 13 or i >= 13:
            print('bug')
        best_iou = 0
        best_anchor = 0
        for k, anchor in enumerate(anchors):
            # Find IOU between box shifted to origin and anchor box.
            box_maxes = box[2:4] / 2.
            box_mins = -box_maxes
            anchor_maxes = (anchor / 2.)
            anchor_mins = -anchor_maxes

            intersect_mins = np.maximum(box_mins, anchor_mins)
            intersect_maxes = np.minimum(box_maxes, anchor_maxes)
            intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
            intersect_area = intersect_wh[0] * intersect_wh[1]
            box_area = box[2] * box[3]
            anchor_area = anchor[0] * anchor[1]
            iou = intersect_area / (box_area + anchor_area - intersect_area)
            if iou > best_iou:
                best_iou = iou
                best_anchor = k

        if best_iou > 0:
            print(i, j, best_anchor)
            detectors_mask[i, j, best_anchor] = 1
            adjusted_box = np.array(
                [
                    box[0] - j, box[1] - i,
                    np.log(box[2] / anchors[best_anchor][0]),
                    np.log(box[3] / anchors[best_anchor][1]), box_class
                ],
                dtype=np.float32)
            matching_true_boxes[i, j, best_anchor] = adjusted_box
    return detectors_mask, matching_true_boxes

In [10]:
# Read anchor_5.txt
anchors = []
with open('/Volumes/JS/UECFOOD100_JS/generated_anchors/anchors_5.txt', 'r') as anchor_file:
    for i, line in enumerate(anchor_file):
        line = line.rstrip('\n')
        anchors.append(list(map(float, line.split(', '))))
anchors = np.array(anchors)
print('-> anchors acquired\n')
print(anchors)

txt_path = '/Volumes/JS/UECFOOD100_JS/train_uec100.txt'
img_size = np.array([800, 600])

# Generate dictionary with labels and ids (not necessary)
label_dict = create_label_dict('/Volumes/JS/UECFOOD100_JS/classes.txt')
print(label_dict)

# Convert txt info to data
image_data = txt2data(txt_path)
images = load_images(image_data)
images2npv(images, image_data)

print('Done!')

-> anchors acquired

[[ 4.46  3.54]
 [ 6.5   5.89]
 [ 9.42  7.84]
 [11.3   5.39]
 [12.38  8.96]]

-> creating dictinary for labels...

{'rice': 0, 'eels on rice': 1, 'pilaf': 2, "chicken-'n'-egg on rice": 3, 'pork cutlet on rice': 4, 'beef curry': 5, 'sushi': 6, 'chicken rice': 7, 'fried rice': 8, 'tempura bowl': 9, 'bibimbap': 10, 'toast': 11, 'croissant': 12, 'roll bread': 13, 'raisin bread': 14, 'chip butty': 15, 'hamburger': 16, 'pizza': 17, 'sandwiches': 18, 'udon noodle': 19, 'tempura udon': 20, 'soba noodle': 21, 'ramen noodle': 22, 'beef noodle': 23, 'tensin noodle': 24, 'fried noodle': 25, 'spaghetti': 26, 'Japanese-style pancake': 27, 'takoyaki': 28, 'gratin': 29, 'sauteed vegetables': 30, 'croquette': 31, 'grilled eggplant': 32, 'sauteed spinach': 33, 'vegetable tempura': 34, 'miso soup': 35, 'potage': 36, 'sausage': 37, 'oden': 38, 'omelet': 39, 'ganmodoki': 40, 'jiaozi': 41, 'stew': 42, 'teriyaki grilled fish': 43, 'fried fish': 44, 'grilled salmon': 45, 'salmon meuniere '

7 6 4
6 6 4
7 4 2
5 6 4
2 5 3
5 2 0
6 6 4
6 6 4
6 6 4
7 6 4
6 6 4
6 8 2
6 5 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 2
6 6 4
6 6 4
7 6 4
6 6 4
5 6 4
5 6 4
1 3 0
6 6 4
6 6 4
6 6 4
6 6 4
5 7 3
7 6 4
6 6 4
6 6 4
7 6 4
5 6 4
6 6 4
5 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
7 6 4
6 6 4
6 6 2
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 7 2
7 6 4
6 6 4
6 6 4
5 6 4
6 7 2
7 6 4
6 6 4
7 6 4
6 6 4
6 3 2
6 8 2
6 5 4
5 6 4
6 5 1
6 6 4
6 6 4
6 5 4
6 8 2
5 6 4
6 6 4
6 6 4
6 5 2
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 2
6 6 4
6 10 1
6 3 2
6 6 4
6 6 4
6 6 4
6 7 2
6 6 4
7 4 2
6 6 4
6 6 4
6 6 4
5 11 1
7 5 2
6 6 2
4 6 2
6 5 4
5 6 2
5 6 4
6 6 4
6 6 4
7 8 1
5 7 2
2 3 0
6 6 4
6 6 4
7 6 4
6 5 4
6 6 4
6 6 4
4 6 2
4 11 0
7 5 2
4 6 4
6 6 4
5 6 2
7 10 0
4 10 0
9 3 2
6 6 2
7 8 2
5 5 4
6 6 4
4 6 4
6 6 4
8 3 1
4 2 0
7 9 2
6 6 4
7 6 1
7 5 4
6 6 4
6 6 4
7 3 1
5 4 0
4 5 1
4 6 1
8 9 2
3 2 1
7 7 4
6 5 1
1 2 0
6 6 2
8 3 2
4 4 0
2 6 0
6 6 2
5 3 1
1 2 0
4 4 1
3 6 1
5 9 1
1 0 0
7 5 2
6 4 1
6 9 1
6 6 4
6 5 1
2 2 1
2 3 0
2 5 0
6 6 2
4 4 1
3 5 0

7 3 1
7 9 1
7 4 2
4 5 2
9 9 1
6 2 1
7 8 2
8 8 2
6 6 4
9 2 1
3 2 1
6 7 0
7 9 0
6 9 1
5 4 0
2 8 1
7 6 4
5 7 2
6 6 2
6 7 4
2 7 0
3 9 0
8 9 0
8 8 1
6 8 1
6 2 1
6 6 4
7 7 2
7 6 4
6 6 4
6 6 4
6 7 2
6 9 1
3 6 1
6 5 4
6 6 4
6 6 4
6 6 4
6 6 4
2 7 1
4 3 1
8 9 1
5 6 4
3 8 1
9 7 0
9 7 0
4 8 1
6 6 4
8 2 1
7 9 1
6 6 4
6 6 4
6 5 2
4 7 0
6 9 1
7 3 1
4 5 2
9 7 1
4 7 0
6 6 4
10 7 1
4 4 1
4 4 1
5 10 1
5 11 1
9 8 1
9 6 3
3 6 1
9 2 1
9 10 0
6 7 0
6 6 4
5 10 1
7 4 2
7 3 1
6 9 1
4 10 1
6 2 1
6 9 1
6 4 2
6 2 1
1 3 0
6 11 1
7 9 1
5 5 4
7 7 2
6 6 4
6 6 4
5 2 1
6 9 2
5 6 2
7 2 1
6 10 1
4 2 1
6 9 1
7 3 2
6 9 1
2 8 0
8 8 1
6 6 4
7 9 2
6 6 1
6 6 4
6 5 2
6 5 2
6 6 4
6 2 1
6 11 0
4 8 0
8 6 1
4 10 1
6 7 2
5 6 2
10 8 1
6 6 4
4 8 2
5 2 0
6 4 2
6 5 4
3 4 1
8 10 1
3 3 1
9 2 1
2 8 0
6 6 4
7 11 0
3 8 1
6 6 4
6 4 2
10 7 1
5 6 2
6 6 4
6 6 4
2 8 0
8 9 1
4 2 0
6 6 4
7 2 1
5 9 1
9 9 1
3 7 1
9 4 1
6 6 2
3 3 1
9 6 1
7 2 1
6 9 2
6 6 4
7 3 1
7 9 1
6 6 2
6 6 4
3 10 0
7 2 1
8 6 0
5 5 0
7 4 0
8 10 1
7 4 2
6 6 2
6 6 2
7 6 4
9 4 1
9 9 1


6 6 4
6 6 4
5 6 4
6 6 4
5 6 4
6 6 4
6 6 4
7 6 4
6 6 4
5 6 4
6 6 4
5 6 4
6 6 4
8 6 4
6 6 4
6 6 4
6 6 4
5 6 2
7 6 4
5 6 4
6 6 4
5 6 4
7 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
7 6 4
6 6 4
6 6 2
4 6 4
6 6 4
5 6 4
7 6 4
8 6 4
6 5 4
6 6 4
5 6 4
5 7 4
6 6 4
6 6 4
6 6 4
6 6 4
4 6 4
6 6 4
9 6 4
6 6 4
6 6 4
6 6 4
5 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 8 2
5 6 3
6 6 4
6 6 4
3 6 1
6 6 2
5 7 4
6 3 1
4 10 0
5 1 1
5 2 1
2 9 0
9 10 0
3 7 0
7 6 4
7 6 2
8 9 1
8 8 1
9 5 2
6 6 4
6 3 1
8 3 1
8 3 2
8 6 4
8 3 1
7 6 4
5 6 4
6 6 4
5 6 4
6 6 4
6 7 4
6 6 4
6 6 4
6 6 3
6 6 4
6 6 4
6 6 4
2 4 1
6 6 4
6 7 4
8 9 1
4 7 0
6 6 4
6 6 4
9 4 2
7 6 4
6 6 4
6 6 4
7 6 2
6 6 4
6 6 4
6 6 4
6 7 4
6 5 4
6 6 1
6 6 2
6 7 2
6 6 4
6 7 2
6 6 1
6 6 2
6 6 4
9 6 1
6 6 4
4 3 2
9 9 2
7 7 4
6 7 2
6 7 4
5 6 2
5 5 2
5 6 4
6 8 2
5 5 4
6 6 4
8 7 1
2 6 0
6 6 4
6 6 4
5 6 4
6 6 4
6 6 2
7 6 4
6 6 4
7 6 4
7 6 2
8 4 1
5 10 0
3 3 0
6 6 4
7 6 1
7 6 4
8 8 2
8 5 1
7 8 2
6 6 4
5 5 2
6 6 4
7 6 1
7 8 2
5 5 1
6 6 4
7 6 2
5 9 2
7 5 2
6 6 4
6 6 4
6

8 6 3
9 8 3
5 6 4
7 6 4
6 9 1
5 5 4
7 6 4
5 5 1
7 6 4
5 6 4
8 6 0
8 6 3
8 6 3
8 5 3
7 7 2
6 6 4
7 6 2
8 6 3
7 6 4
8 5 4
6 6 2
7 6 3
5 6 3
7 6 3
5 6 2
8 5 2
7 5 2
6 7 2
7 6 2
7 6 3
6 5 4
5 6 4
9 6 3
6 6 4
6 6 2
6 6 4
8 6 4
5 6 4
5 6 2
6 6 4
6 7 4
6 6 4
5 6 4
6 6 4
6 6 4
6 6 4
7 6 4
6 6 4
6 2 1
4 6 1
7 7 4
6 6 4
6 6 4
5 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 2
6 6 4
6 6 4
7 6 4
6 6 4
6 6 4
6 3 2
6 6 2
6 7 1
6 6 2
10 6 0
6 6 4
3 6 1
5 8 1
6 6 4
5 6 4
4 7 2
10 4 3
4 2 1
7 5 4
6 6 2
5 6 4
7 6 4
6 6 2
6 6 4
6 6 4
6 6 4
7 7 2
6 5 2
6 8 2
2 8 1
6 6 4
6 6 4
6 6 4
8 8 2
1 5 0
7 7 2
7 6 2
2 7 0
6 4 1
6 6 4
6 6 4
7 7 2
7 6 4
6 6 4
6 6 4
6 6 2
5 6 2
6 6 4
6 6 4
6 6 4
10 6 3
4 6 3
5 6 4
7 6 4
2 5 3
5 6 3
6 5 4
6 6 4
6 6 4
6 7 0
7 4 1
4 5 0
6 6 4
6 6 4
5 6 3
6 6 4
5 6 3
6 8 2
6 6 3
6 6 2
4 9 1
6 6 4
6 6 4
7 6 3
6 6 3
5 6 4
6 6 3
3 8 3
4 8 3
8 5 3
6 6 3
6 6 4
5 6 4
3 5 3
6 6 4
6 6 3
9 6 3
6 6 4
6 6 4
6 5 4
6 6 4
6 6 3
6 5 4
5 6 4
6 6 3
6 6 4
6 6 4
6 5 2
7 6 3
9 5 3
6 4 2
2 10 0
6 6 3
6 6 3
6 7 4
9 6 3


8 5 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
5 6 4
6 6 4
7 6 4
6 6 4
5 6 4
7 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 4
6 6 2
5 4 2
6 6 4
7 7 4
5 6 4
6 6 4
6 7 4
4 6 4
6 6 4
7 6 4
4 4 2
9 6 2
6 5 4
5 6 4
6 6 4
8 6 4
5 6 4
6 6 4
6 6 4
6 6 4
6 6 4
8 7 4
6 6 4
9 7 2
9 6 3
6 7 2
8 6 4
7 6 4
6 6 4
6 6 4
6 6 4
6 6 4
7 5 2
7 6 4
6 5 4
6 6 4
4 7 4
6 6 4
5 6 1
6 8 2
6 5 2
5 7 1
7 4 1
6 6 4
8 6 4
7 6 4
7 6 4
6 6 4
5 8 2
6 6 2
6 6 2
7 6 4
6 6 4
7 6 4
5 6 4
6 6 4
6 6 4
6 7 4
7 6 4
6 6 4
7 6 4
5 6 4
7 6 4
5 6 4
5 6 2
5 7 2
6 6 3
6 6 3
6 6 3
6 6 4
6 6 4
6 6 4
7 6 3
6 6 4
7 5 4
6 6 4
6 6 4
7 5 4
6 4 2
6 6 1
6 6 4
7 6 4
7 7 4
6 6 4
6 6 4
7 6 4
6 6 4
6 5 2
7 6 4
6 6 4
6 6 4
6 6 4
7 6 4
6 6 4
8 6 4
6 6 2
7 6 4
6 6 4
6 7 3
6 6 4
6 6 2
5 6 2
5 6 4
5 6 2
6 7 4
7 6 2
6 6 4
6 6 2
8 8 2
6 6 4
6 6 4
6 6 4
5 6 2
5 6 2
6 6 4
6 6 4
5 6 4
8 6 3
6 7 2
5 6 4
5 6 4
7 6 4
7 6 4
6 6 4
6 6 2
7 6 4
5 6 4
6 6 4
6 6 4
7 6 4
6 6 4
6 5 4
6 6 4
6 6 4
6 6 4
5 6 4
6 6 4
6 6 4
6 6 4
7 6 4
6 6 

6 6 4
6 6 4
9 6 3
6 6 4
5 6 4
6 6 4
6 5 4
6 6 1
6 9 2
6 6 4
7 6 4
6 7 4
4 4 0
6 6 2
6 6 2
6 6 2
5 6 2
5 6 2
6 7 1
5 5 1
6 5 4
6 6 4
6 6 4
5 4 1
6 6 3
6 6 4
6 6 4
8 3 1
6 6 4
5 5 4
6 7 2
7 6 2
6 7 1
5 5 1
6 6 1
6 6 2
6 6 2
5 6 2
5 6 3
6 6 1
6 7 2
5 8 2
6 6 4
6 6 4
4 6 0
7 6 1
5 6 4
6 6 4
5 6 4
5 6 4
6 6 4
5 6 2
6 6 4
6 6 4
6 6 4
6 5 4
6 6 4
6 6 4
3 6 0
4 10 0
9 2 1
7 6 4
6 6 4
6 6 4
6 6 3
5 6 1
6 6 1
5 6 4
6 6 4
6 6 2
5 6 1
5 6 1
5 6 1
6 6 4
6 8 1
6 6 2
5 4 2
6 6 4
4 4 1
6 6 4
6 6 4
4 5 2
5 6 2
7 6 4
6 5 2
6 6 4
5 6 4
6 6 4
5 6 2
6 6 4
5 6 4
6 7 4
6 6 1
6 6 2
6 6 4
6 6 2
5 6 2
6 6 3
4 5 2
5 7 1
5 6 3
5 6 2
6 7 2
6 6 3
5 6 4
5 6 1
6 6 4
5 6 4
7 6 4
7 7 4
6 6 2
7 6 4
5 6 3
6 6 3
6 6 4
6 6 3
5 6 4
5 6 4
7 6 4
6 6 4
5 6 4
6 5 2
5 5 2
5 5 2
5 6 2
6 6 3
7 6 2
6 6 4
5 6 2
8 6 3
5 6 2
7 6 4
6 6 4
6 6 4
8 6 2
5 6 3
5 6 4
6 6 2
6 6 3
6 6 4
8 6 4
6 6 3
4 6 3
5 6 4
6 6 4
8 5 2
5 6 3
8 6 4
5 6 4
6 6 4
7 5 3
6 6 3
7 6 3
7 5 2
6 6 2
7 4 2
5 6 2
4 8 1
5 6 4
6 6 2
6 7 2
5 6 1
6 6 4
7 6 3
7 6 4
7 6 4
6 6